In [1]:
# All imports

import zoopla # Python wrapper for Zoopla API. Installed through pip.
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.6f' % x,'display.max_columns', None, 'display.max_rows', None)

# To get latest file in directory
import glob
import os

import datetime
from time import sleep
import googlemaps # Python wrapper for Google Maps API. Installed through pip.
import time

In [2]:
# t can only be 8 days in the past.
t = datetime.datetime(2018,4,16,9)
print t

2018-04-16 09:00:00


In [3]:
def distance_work_1(address):
    directions_result = gmaps.directions(address,
                                         "...",
                                         mode="transit",
                                         arrival_time=t)
    try:
        a = directions_result[0]["legs"][0]["duration"]["text"]
        return convert_to_mins(a)
    except IndexError:
        print "Address empty             ", address

In [ ]:
def distance_work_2(address):
    directions_result = gmaps.directions(address,
                                         "...",
                                         mode="transit",
                                         arrival_time=t)
    try:
        a = directions_result[0]["legs"][0]["duration"]["text"]
        return convert_to_mins(a)
    except IndexError:
        print "Address empty             ", address

In [ ]:
def cycling_work_2(address):
    directions_result = gmaps.directions(address,
                                         "...",
                                         mode="bicycling",
                                         arrival_time=t)
    try:
        a = directions_result[0]["legs"][0]["duration"]["text"]
        return convert_to_mins(a)
    except IndexError:
        print "Address empty             ", address

In [ ]:
def convert_to_mins(s):
    if len(str(s).split(" "))==2:
        return int(s.split(" ")[0])
    if len(str(s).split(" "))==4:
        return int(s.split(" ")[0])*60 + int(s.split(" ")[2])

In [ ]:
"""
Google Maps API's
"""

gmapsAPI = {'API_1':'...', 'API_2':'...'}

In [11]:
"""
Adds commute times
"""

# Reads file from previous run
list_of_files = glob.glob('.../Zoopla/*')
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file, index_col=0)
df = df.transpose()
l = df.to_dict()

c = 0
written = 0
passed = 0

a = 0 # API key list iterator
a_list = ['gmaps_API_1', 'gmaps_API_2']
gmaps = googlemaps.Client(key=gmapsAPI[a_list[a]])

class GetOutOfLoop( Exception ):
    pass

t0 = time.time()

try:
    for i in l.keys():
        print c
        c+=1
        address = l[i]['displayable_address']
        length = len(address.split(","))
        while True:
            try:
                if any( [ l[i]['distance_work_1'] != l[i]['distance_work_1'], 
                          l[i]['distance_work_2'] != l[i]['distance_work_2'],
                          l[i]['distance_work_1'] == l[i]['distance_work_2'],
                          any(x not in l[i].keys() for x in ['distance_work_1','distance_work_2']) ] ):
                    if l[i]['distance_work_1'] == l[i]['distance_work_2']:
                        for x in range(length):
                            try:
                                l[i]['distance_work_2'] = distance_work_2(','.join(address.split(",")[x:]))
                            except (googlemaps.exceptions.ApiError, IndexError, TypeError) as e:
                                print "Address failed:          ", ','.join(address.split(",")[x:])
                            else:
                                written += 1
                                break
                    else:
                        for x in range(length):
                            try:
                                l[i]['distance_work_1'] = distance_work_1(','.join(address.split(",")[x:]))
                            except (googlemaps.exceptions.ApiError, IndexError, TypeError) as e:
                                print "Address failed:          ", ','.join(address.split(",")[x:])
                            else:
                                l[i]['distance_work_2'] = distance_work_2(','.join(address.split(",")[x:]))
                                written += 1
                                break
                else:
                    passed += 1
     
                break
        
            except (googlemaps.exceptions.Timeout, googlemaps.exceptions.TransportError, googlemaps.exceptions.HTTPError) as e:
                if a <= 3:
                    a += 1
                    gmaps = googlemaps.Client(key=gmapsAPI[a_list[a]])
                    print "Changed API key to: ", a_list[a]
                else:
                    raise GetOutOfLoop
    raise GetOutOfLoop
                
except GetOutOfLoop:
    print "API limits reached"
    t1 = time.time()
    total = t1-t0
    total_length = len(l.keys())
    print "Minutes: ", total/60
    print "Length of dictionary: ", total_length
    print "Length covered: ", c
    print "Passed: ", passed
    print "Written: ", written 
    df = df.from_dict(l)
    df = df.transpose()
    now = datetime.datetime.now().strftime("%H%M%d%m%y")
    df.to_csv(".../gmaps/"+str(now)+".csv",encoding='utf-8')

0
1
2
3
4
5
Address failed:           Orchestra Court, Edgware, Middlesex HA8
6



**Now I need to filter down to less than 60 minutes each.**

In [12]:
list_of_files = glob.glob('.../gmaps/*')
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file,index_col=0)
d4 = df.copy()
d4['distance_work_2'] = pd.to_numeric(d4['distance_work_2'])
d4['distance_work_1'] = pd.to_numeric(d4['distance_work_1'])
d4 = d4[(d4['distance_work_2']<=60)&(d4['distance_work_1']<=60)]
d4.shape
now = datetime.datetime.now().strftime("%H%M%d%m%y")
d4.to_csv(".../filtered/"+str(now)+".csv",encoding='utf-8')